In [1]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
!pip install missingno
!pip install sklearn.cross_validation

     - 43.9MB 102.4MB/s
     |████████████████████████████████| 81kB 6.5MB/s 
     |████████████████████████████████| 604kB 18.6MB/s 
     |████████████████████████████████| 3.1MB 44.0MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 296kB 52.0MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.10.0-py2.py3-none-any.whl size=232425 sha256=e6addc16f91a119518831e5106c5d9b47933374bda120bc8074240443cd904db
  Stored in directory: /tmp/pip-ephem-wheel-cache-xliyj34c/wheels/56/c2/dd/8d945b0443c35df7d5f62fa9e9ae105a2d8b286302b92e0109
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27085 sha256=058a2356f9bf2aa256c50a44135c2eae1d396fde1e377cb49c105051ed6ff408
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
Successfully built pandas-profiling htmlmin
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully u

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import missingno as msno
import pandas_profiling
import seaborn as sn
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
from plotly.offline import iplot
import seaborn as sns
import matplotlib.gridspec as gridspec
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,recall_score,precision_recall_curve,auc,roc_curve,roc_auc_score,classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from pandas import DataFrame

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.



In [3]:
creditcard = pd.read_csv('/content/drive/My Drive/Case Study/creditcard.csv')

In [4]:
# get ride of duplicate rows
creditcard = creditcard.drop_duplicates()

In [5]:
#scaling
rob_scaler = RobustScaler()

creditcard['scaled_amount'] = rob_scaler.fit_transform(creditcard['Amount'].values.reshape(-1,1))
creditcard['scaled_time'] = rob_scaler.fit_transform(creditcard['Time'].values.reshape(-1,1))

creditcard.drop(['Time','Amount'], axis=1, inplace=True)

In [6]:
creditcard_df = creditcard.copy()

In [7]:
def data_prepration(x): 
    x_features= x.iloc[:,x.columns != "Class"]
    x_labels=x.iloc[:,x.columns=="Class"]
    x_features_train,x_features_test,x_labels_train,x_labels_test = train_test_split(x_features,x_labels,random_state=42,test_size=0.2)

    return(x_features_train,x_features_test,x_labels_train,x_labels_test)

In [8]:
clf= RandomForestClassifier(n_estimators=100)

In [9]:
os = SMOTE(random_state=0) #   We are using SMOTE as the function for oversampling
# now we can devided our data into training and test data
# Call our method data prepration on our dataset
data_train_X,data_test_X,data_train_y,data_test_y=data_prepration(creditcard_df)
columns = data_train_X.columns

In [10]:
os_data_X,os_data_y=os.fit_sample(data_train_X,data_train_y)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns)
os_data_y= pd.DataFrame(data=os_data_y,columns=["Class"])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of normal transcation in oversampled data",len(os_data_y[os_data_y["Class"]==0]))
print("Number of fraud transcation",len(os_data_y[os_data_y["Class"]==1]))
print("Proportion of Normal data in oversampled data is ",len(os_data_y[os_data_y["Class"]==0])/len(os_data_X))
print("Proportion of fraud data in oversampled data is ",len(os_data_y[os_data_y["Class"]==1])/len(os_data_X))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



length of oversampled data is  453194
Number of normal transcation in oversampled data 226597
Number of fraud transcation 226597
Proportion of Normal data in oversampled data is  0.5
Proportion of fraud data in oversampled data is  0.5


In [11]:
df_test = data_test_X.copy()
df_test["Class"] = data_test_y["Class"]
df_test

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,scaled_amount,scaled_time,Class
104697,1.054379,-0.764756,0.160168,0.665587,1.113466,4.771656,-1.536609,1.306483,1.565054,-0.349941,-1.238496,0.608002,-0.249115,-0.933969,-1.407873,-0.191383,-0.180853,0.226980,0.537470,0.050995,-0.097138,-0.029759,-0.171596,1.047201,0.698595,-0.217939,0.101970,0.040152,0.478654,-0.182124,0
134356,-4.805134,4.351191,-0.916135,-0.900752,-0.870777,-0.782946,0.328104,0.582501,2.749336,4.728060,0.455162,0.649740,0.930379,-1.663325,1.334738,-0.283321,0.152340,-0.950771,-0.935567,2.160193,-0.854486,-0.758266,0.147005,-0.021274,0.552660,0.008318,0.431835,-0.436239,-0.169378,-0.046108,0
42637,-1.549833,-0.261143,1.556289,-2.037817,-0.366315,-1.334314,0.165406,-0.025782,-1.472751,-0.172838,-0.594842,0.063059,1.054185,-0.171535,0.192041,-0.867483,-0.348274,0.743352,-1.454672,-0.347365,-0.405656,-0.951005,-0.147752,0.424100,0.081845,1.103559,-0.502271,-0.248822,0.334863,-0.511327,0
882,0.216344,0.663182,1.303520,0.169219,-0.404266,-0.517288,-0.036938,-0.838221,-0.528675,-0.174777,1.648466,1.114947,0.202150,0.479424,0.226420,0.456003,-0.558258,-0.101939,0.000758,-0.184929,0.552788,-0.961206,-0.014642,0.525962,0.750877,0.141543,0.045107,0.127321,-0.256154,-0.987464,0
263275,-0.273365,0.825649,0.555674,0.384915,1.343842,1.984159,0.321808,0.510275,-0.891463,-0.115561,0.952916,-0.221254,-0.867646,0.974855,2.441035,-1.711023,0.902400,0.022272,2.097232,0.172188,0.346482,1.162889,-0.557535,-0.896146,0.503553,0.555129,0.129099,0.047571,-0.266444,0.895224,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154645,-0.770202,0.837344,2.577987,4.730948,0.022972,0.723626,0.000571,0.207992,-0.391169,0.884210,-0.413787,-4.293800,0.252938,1.787482,0.787452,0.322678,0.803232,1.146681,0.747173,0.388969,0.128625,0.313392,-0.053439,-0.066768,0.113695,0.531149,0.037346,0.101904,1.026839,0.207719,0
107561,1.051961,-0.835149,0.801527,0.700204,-1.389168,-0.364323,-0.584358,-0.057858,-0.425812,0.644106,-0.959899,-0.088502,0.089207,-0.194018,0.349357,-1.424226,-0.046620,1.472272,-1.153242,-0.299870,-0.168762,-0.139388,-0.160697,0.378303,0.471809,-0.235528,0.052804,0.055817,1.571409,-0.166823,0
185406,1.976646,-0.307673,-0.943425,0.038003,-0.226856,-0.677663,-0.151479,-0.049804,0.446497,0.268931,0.680633,0.403169,-0.842490,0.702412,0.283798,0.786716,-0.908998,-0.067118,0.452939,-0.177760,-0.426105,-1.386126,0.478793,-0.482260,-0.742871,0.025402,-0.083591,-0.059167,0.152691,0.493065,0
59492,-2.847982,-2.335998,1.712881,-0.237077,0.321139,-0.790762,0.243713,-0.292275,1.126027,-0.158992,0.992291,0.964422,-0.132054,-0.947165,-1.481059,0.315601,-0.598059,-0.175970,0.383804,-1.496600,-0.551132,0.441075,1.883552,0.614984,-0.532756,0.694734,0.258013,-0.358734,0.970102,-0.421038,0


In [12]:
clf.fit(os_data_X,os_data_y.values.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
pred=clf.predict(data_test_X)

In [18]:
df_test['predicted_probability'] = clf.predict_proba(data_test_X)[:, 1]
df_test.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,scaled_amount,scaled_time,Class,predicted_probability
104697,1.054379,-0.764756,0.160168,0.665587,1.113466,4.771656,-1.536609,1.306483,1.565054,-0.349941,-1.238496,0.608002,-0.249115,-0.933969,-1.407873,-0.191383,-0.180853,0.226980,0.537470,0.050995,-0.097138,-0.029759,-0.171596,1.047201,0.698595,-0.217939,0.101970,0.040152,0.478654,-0.182124,0,0.00
134356,-4.805134,4.351191,-0.916135,-0.900752,-0.870777,-0.782946,0.328104,0.582501,2.749336,4.728060,0.455162,0.649740,0.930379,-1.663325,1.334738,-0.283321,0.152340,-0.950771,-0.935567,2.160193,-0.854486,-0.758266,0.147005,-0.021274,0.552660,0.008318,0.431835,-0.436239,-0.169378,-0.046108,0,0.01
42637,-1.549833,-0.261143,1.556289,-2.037817,-0.366315,-1.334314,0.165406,-0.025782,-1.472751,-0.172838,-0.594842,0.063059,1.054185,-0.171535,0.192041,-0.867483,-0.348274,0.743352,-1.454672,-0.347365,-0.405656,-0.951005,-0.147752,0.424100,0.081845,1.103559,-0.502271,-0.248822,0.334863,-0.511327,0,0.01
882,0.216344,0.663182,1.303520,0.169219,-0.404266,-0.517288,-0.036938,-0.838221,-0.528675,-0.174777,1.648466,1.114947,0.202150,0.479424,0.226420,0.456003,-0.558258,-0.101939,0.000758,-0.184929,0.552788,-0.961206,-0.014642,0.525962,0.750877,0.141543,0.045107,0.127321,-0.256154,-0.987464,0,0.00
263275,-0.273365,0.825649,0.555674,0.384915,1.343842,1.984159,0.321808,0.510275,-0.891463,-0.115561,0.952916,-0.221254,-0.867646,0.974855,2.441035,-1.711023,0.902400,0.022272,2.097232,0.172188,0.346482,1.162889,-0.557535,-0.896146,0.503553,0.555129,0.129099,0.047571,-0.266444,0.895224,0,0.02


In [25]:
df_test['proba_decile'] = pd.qcut(df_test['predicted_probability'].rank(method='first'), q=20, labels=False)
A, bins = pd.qcut(df_test['predicted_probability'].rank(method='first'), q=20, labels=False, retbins = True)

bins
df_test.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,scaled_amount,scaled_time,Class,predicted_probability,proba_decile
104697,1.054379,-0.764756,0.160168,0.665587,1.113466,4.771656,-1.536609,1.306483,1.565054,-0.349941,-1.238496,0.608002,-0.249115,-0.933969,-1.407873,-0.191383,-0.180853,0.226980,0.537470,0.050995,-0.097138,-0.029759,-0.171596,1.047201,0.698595,-0.217939,0.101970,0.040152,0.478654,-0.182124,0,0.00,0
134356,-4.805134,4.351191,-0.916135,-0.900752,-0.870777,-0.782946,0.328104,0.582501,2.749336,4.728060,0.455162,0.649740,0.930379,-1.663325,1.334738,-0.283321,0.152340,-0.950771,-0.935567,2.160193,-0.854486,-0.758266,0.147005,-0.021274,0.552660,0.008318,0.431835,-0.436239,-0.169378,-0.046108,0,0.01,17
42637,-1.549833,-0.261143,1.556289,-2.037817,-0.366315,-1.334314,0.165406,-0.025782,-1.472751,-0.172838,-0.594842,0.063059,1.054185,-0.171535,0.192041,-0.867483,-0.348274,0.743352,-1.454672,-0.347365,-0.405656,-0.951005,-0.147752,0.424100,0.081845,1.103559,-0.502271,-0.248822,0.334863,-0.511327,0,0.01,17
882,0.216344,0.663182,1.303520,0.169219,-0.404266,-0.517288,-0.036938,-0.838221,-0.528675,-0.174777,1.648466,1.114947,0.202150,0.479424,0.226420,0.456003,-0.558258,-0.101939,0.000758,-0.184929,0.552788,-0.961206,-0.014642,0.525962,0.750877,0.141543,0.045107,0.127321,-0.256154,-0.987464,0,0.00,0
263275,-0.273365,0.825649,0.555674,0.384915,1.343842,1.984159,0.321808,0.510275,-0.891463,-0.115561,0.952916,-0.221254,-0.867646,0.974855,2.441035,-1.711023,0.902400,0.022272,2.097232,0.172188,0.346482,1.162889,-0.557535,-0.896146,0.503553,0.555129,0.129099,0.047571,-0.266444,0.895224,0,0.02,18


In [35]:
df_test["predicted_probability"].unique()


array([0.  , 0.01, 0.02, 0.03, 0.04, 0.06, 0.05, 0.95, 0.11, 1.  , 0.1 ,
       0.07, 0.17, 0.09, 0.56, 0.08, 0.15, 0.63, 0.91, 0.18, 0.16, 0.97,
       0.13, 0.21, 0.14, 0.12, 0.69, 0.87, 0.92, 0.98, 0.88, 0.19, 0.29,
       0.39, 0.67, 0.7 , 0.99, 0.31, 0.8 , 0.86, 0.28, 0.26, 0.94, 0.25,
       0.33, 0.46, 0.93, 0.41, 0.23, 0.5 , 0.24, 0.96, 0.22, 0.35, 0.2 ,
       0.52, 0.3 , 0.89, 0.75, 0.53, 0.48])